In [49]:
from enum import Enum, IntEnum
from prettytable import PrettyTable


class FunctionalUnit(Enum):
    Integer = 1
    Mult1 = 2
    Mult2 = 3
    Add = 4
    Divide = 5

FloatRegister = IntEnum("FloatRegister", ['F0','F1','F2','F3','F4','F5','F6','F7','F8'
                                ,'F9','F10','F11','F12','F13','F14','F15',
                                'F16','F17','F18','F19','F20','F21','F22',
                                'F23','F24','F25','F26','F27','F28','F29',
                                'F30','F31'], start = 1)

Register = IntEnum("Register", ['R0','R1','R2','R3','R4','R5','R6','R7','R8'
                                ,'R9','R10','R11','R12','R13','R14','R15',
                                'R16','R17','R18','R19','R20','R21','R22',
                                'R23','R24','R25','R26','R27','R28','R29',
                                'R30','R31'], start = 1)

class FunctionalEntry:
    def __init__(self) -> None:
        self.time = -1
        self.busy: bool = False
        self.op: str = None
        self.dst: FloatRegister | Register = None
        self.s1 : FloatRegister | Register = None
        self.s2 : FloatRegister | Register = None
        self.s1_fu: FunctionalUnit = None
        self.s2_fu: FunctionalUnit = None
        self.s1_ready: bool = True
        self.s2_ready: bool = True
        self.instr: Instruction = None
    
    def clear(self):
        self.time = -1
        self.busy: bool = False
        self.op: str = None
        self.dst: FloatRegister | Register = None
        self.s1 : FloatRegister | Register = None
        self.s2 : FloatRegister | Register = None
        self.s1_fu: FunctionalUnit = None
        self.s2_fu: FunctionalUnit = None
        self.s1_ready: bool = True
        self.s2_ready: bool = True
        self.instr: Instruction = None

class FunctionalUnitStatus:
    def __init__(self) -> None:
        self.table: dict[FunctionalUnit, FunctionalEntry] = {FunctionalUnit.Integer: FunctionalEntry(),    \
                                                      FunctionalUnit.Mult1: FunctionalEntry(),       \
                                                      FunctionalUnit.Mult2: FunctionalEntry(),       \
                                                      FunctionalUnit.Add: FunctionalEntry(),         \
                                                      FunctionalUnit.Divide: FunctionalEntry() }
    def display_table(self):
        
        def name_ok(obj) -> str:
            if not obj:
                return ""
            return obj.name
        
        table = PrettyTable(["Time", "Name", "Busy", "Op", "Fi", "Fj", "Fk", "Qj", "Qk", "Rj", "Rk"])
        for (fu, fu_entry) in self.table.items():
            table.add_row([str(fu_entry.time) if fu_entry.time != -1 else "" , name_ok(fu), str(fu_entry.busy), fu_entry.op if fu_entry.op else "", name_ok(fu_entry.dst),  name_ok(fu_entry.s1), name_ok(fu_entry.s2), name_ok(fu_entry.s1_fu), name_ok(fu_entry.s2_fu), str(fu_entry.s1_ready) if fu_entry.busy else "", str(fu_entry.s2_ready) if fu_entry.busy else ""])
        print(table)
        
class RegisterResultStatus:
    def __init__(self) -> None:
        self.table: list[FunctionalUnit] = [None for _ in range(32)]
        
    def display_table(self):
        l = []
        for i in range(len(self.table)):
            if self.table[i]:
                l.append(i)
        table = PrettyTable(["F{}".format(i) for i in l])
        table.add_row([fu.name for fu in filter(lambda fu: fu != None, self.table)])
        print(table)

class Instruction:
    def __init__(self, name) -> None:
        self.name: str = name
        self.delay = 0
    
    def __str__(self) -> str:
        return self.name

class LD(Instruction):
    def __init__(self, dst: FloatRegister, imm: int, addr: Register) -> None:
        super().__init__("L.D")
        self.dst: FloatRegister = dst
        self.imm: int = imm
        self.addr: Register = addr
        self.delay = 1
    
    def __str__(self) -> str:
        return super().__str__() + " " + "{}, {}({})".format(self.dst.name, self.imm, self.addr.name)

class MULD(Instruction):
    def __init__(self, dst: FloatRegister, src1: FloatRegister, src2: FloatRegister) -> None:
        super().__init__("MUL.D")
        self.dst: FloatRegister = dst
        self.src1: FloatRegister = src1
        self.src2: FloatRegister = src2
        self.delay = 10
    
    def __str__(self) -> str:
        return super().__str__() + " " + "{}, {}, {}".format(self.dst.name, self.src1.name, self.src2.name)
    
class ADDD(Instruction):
    def __init__(self, dst: FloatRegister, src1: FloatRegister, src2: FloatRegister) -> None:
        super().__init__("ADD.D")
        self.dst: FloatRegister = dst
        self.src1: FloatRegister = src1
        self.src2: FloatRegister = src2
        self.delay = 2
    
    def __str__(self) -> str:
        return super().__str__() + " " + "{}, {}, {}".format(self.dst.name, self.src1.name, self.src2.name)
    
class SUBD(Instruction):
    def __init__(self, dst: FloatRegister, src1: FloatRegister, src2: FloatRegister) -> None:
        super().__init__("SUB.D")
        self.dst: FloatRegister = dst
        self.src1: FloatRegister = src1
        self.src2: FloatRegister = src2
        self.delay = 2
    
    def __str__(self) -> str:
        return super().__str__() + " " + "{}, {}, {}".format(self.dst.name, self.src1.name, self.src2.name)
        
class DIVD(Instruction):
    def __init__(self, dst: FloatRegister, src1: FloatRegister, src2: FloatRegister) -> None:
        super().__init__("DIV.D")
        self.dst: FloatRegister = dst
        self.src1: FloatRegister = src1
        self.src2: FloatRegister = src2
        self.delay = 40
    
    def __str__(self) -> str:
        return super().__str__() + " " + "{}, {}, {}".format(self.dst.name, self.src1.name, self.src2.name)

class InstructionStatus:
    def __init__(self) -> None:
        self.issue: int = -1
        self.read_op: int = -1
        self.exec_comp: int = -1
        self.wb: int = -1
    
    def is_wb(self, cycle: int) -> bool:
        return self.exec_comp <= cycle and self.wb == -1

    def is_exec(self, cycle: int) -> bool:
        return self.read_op <= cycle and self.exec_comp == -1
    
    def is_read_op(self, cycle: int) -> bool:
        return self.issue <= cycle and self.read_op == -1
    
    def __str__(self) -> str:
        return "{:6}{:6}{:6}{:6}".format(self.issue,
                                         self.read_op,
                                         self.exec_comp,
                                         self.wb)

def display_instruction_status(instruction_status: dict[Instruction, InstructionStatus]):
    table = PrettyTable(["Instructions", "issue", "read", "exec", "wb"])
    ok = lambda x: str(x) if x != -1 else ""
    for (instr, status) in instruction_status.items():
        table.add_row([str(instr), ok(status.issue), ok(status.read_op), ok(status.exec_comp), ok(status.wb)])
    print(table)
    
instruction_status: dict[Instruction, InstructionStatus] = {}
register_result_status = RegisterResultStatus()
functional_unit_status = FunctionalUnitStatus()

instruction_status_copy: dict[Instruction, InstructionStatus] = {}
register_result_status_copy = RegisterResultStatus()
functional_unit_status_copy = FunctionalUnitStatus()


In [50]:
def parser(instrs: list) -> list[Instruction]:
    res = []
    for instr in instrs:
        if instr[0] == "L.D":
            dst = instr[1]
            imm = instr[2]
            addr = instr[3]
            res.append(LD(dst, imm, addr))
        elif instr[0] == "MUL.D":
            dst = instr[1]
            src1 = instr[2]
            src2 = instr[3]
            res.append(MULD(dst, src1, src2))
        elif instr[0] == "ADD.D":
            dst = instr[1]
            src1 = instr[2]
            src2 = instr[3]
            res.append(ADDD(dst, src1, src2))
        elif instr[0] == "SUB.D":
            dst = instr[1]
            src1 = instr[2]
            src2 = instr[3]
            res.append(SUBD(dst, src1, src2))
        elif instr[0] == "DIV.D":
            dst = instr[1]
            src1 = instr[2]
            src2 = instr[3]
            res.append(DIVD(dst, src1, src2))
    
    return res

instrs = [
    ("L.D", FloatRegister.F6, 34, Register.R2),
    ("L.D", FloatRegister.F2, 45, Register.R3),
    ("MUL.D", FloatRegister.F0, FloatRegister.F2, FloatRegister.F4),
    ("SUB.D", FloatRegister.F8, FloatRegister.F2, FloatRegister.F6),
    ("DIV.D", FloatRegister.F10, FloatRegister.F0, FloatRegister.F6),
    ("ADD.D", FloatRegister.F6, FloatRegister.F8, FloatRegister.F2)
]

instrs: list[Instruction] = parser(instrs)
for instr in instrs:
    instruction_status[instr] = InstructionStatus()

dict_items([(<__main__.LD object at 0x1062ac050>, <__main__.InstructionStatus object at 0x1062ab8d0>), (<__main__.LD object at 0x1062ad2d0>, <__main__.InstructionStatus object at 0x1062adf10>), (<__main__.MULD object at 0x1062af010>, <__main__.InstructionStatus object at 0x1062ade50>), (<__main__.SUBD object at 0x1062ad7d0>, <__main__.InstructionStatus object at 0x1062ace90>), (<__main__.DIVD object at 0x1062add50>, <__main__.InstructionStatus object at 0x1062ad890>), (<__main__.ADDD object at 0x1062aed10>, <__main__.InstructionStatus object at 0x1062ada10>)])

In [51]:
def release_delay(fus: list[FunctionalUnit]):
    for fu in fus:
        fu_entry = functional_unit_status.table[fu]
        for (f, f_entry) in functional_unit_status.table.items():
            if f_entry.s1_fu == fu:
                f_entry.s1_ready = True
            if f_entry.s2_fu == fu:
                f_entry.s2_ready = True
        
        register_result_status.table[fu_entry.dst] = None
        fu_entry.busy = False
        
        fu_entry.clear()


def wb(cycle) -> list[FunctionalUnit]:
    res = []
    for (fu, fu_entry) in functional_unit_status.table.items():
        if not fu_entry.instr:
            continue
        if not instruction_status[fu_entry.instr].is_wb(cycle):
            continue
        if fu_entry.time != 0:
            continue
        
        flag = True
        for (f, f_entry) in functional_unit_status.table.items():
            if not ((f_entry.s1 != fu_entry.dst or f_entry.s1_ready == False) and (f_entry.s2 != fu_entry.dst or f_entry.s2_ready == False)):
                flag = False
        if not flag:
            continue
        instruction_status[fu_entry.instr].wb = cycle
        res.append(fu)
        return res
        # for (f, f_entry) in functional_unit_status.table.items():
        #     if f_entry.s1_fu == fu:
        #         f_entry.s1_ready = True
        #     if f_entry.s2_fu == fu:
        #         f_entry.s2_ready = True
        
        # register_result_status.table[fu_entry.dst] = None
        # fu_entry.busy = False
        
        # fu_entry.clear()
        
        
def exec(cycle):
    for (fu, fu_entry) in functional_unit_status.table.items():
        if not fu_entry.instr:
            continue
        if not instruction_status[fu_entry.instr].is_exec(cycle):
            continue
        
        if fu_entry.time >= 0:
           fu_entry.time -= 1
           
        if fu_entry.time == 0:
            instruction_status[fu_entry.instr].exec_comp = cycle
        
def read_oprands(cycle):
    for (fu, fu_entry) in functional_unit_status.table.items():
        if not fu_entry.instr:
            continue
        if not instruction_status[fu_entry.instr].is_read_op(cycle):
            continue
        
        if not (fu_entry.s1_ready and fu_entry.s2_ready):
            continue
                
        fu_entry.s1_ready, fu_entry.s2_ready = False, False
        fu_entry.s1_fu, fu_entry.s2_fu = None, None
        
        instruction_status[fu_entry.instr].read_op = cycle
        fu_entry.time = fu_entry.instr.delay
        
def issue(instr, cycle) -> bool:
    is_issue = False
    if type(instr) == LD:
        if (not functional_unit_status.table[FunctionalUnit.Integer].busy) and (not register_result_status.table[instr.dst]):
            functional_unit_status.table[FunctionalUnit.Integer].busy = True
            functional_unit_status.table[FunctionalUnit.Integer].instr = instr
            functional_unit_status.table[FunctionalUnit.Integer].op = "load"
            functional_unit_status.table[FunctionalUnit.Integer].dst = instr.dst
            functional_unit_status.table[FunctionalUnit.Integer].s2 = instr.addr
            functional_unit_status.table[FunctionalUnit.Integer].s2_fu = register_result_status.table[instr.addr]
            functional_unit_status.table[FunctionalUnit.Integer].s2_ready = not functional_unit_status.table[FunctionalUnit.Integer].s2_fu
            register_result_status.table[instr.dst] = FunctionalUnit.Integer
            is_issue = True
        
    elif type(instr) == MULD:
        if (not functional_unit_status.table[FunctionalUnit.Mult1].busy) and (not register_result_status.table[instr.dst]):
            functional_unit_status.table[FunctionalUnit.Mult1].busy = True
            functional_unit_status.table[FunctionalUnit.Mult1].instr = instr
            functional_unit_status.table[FunctionalUnit.Mult1].op = "mult"
            functional_unit_status.table[FunctionalUnit.Mult1].dst = instr.dst
            functional_unit_status.table[FunctionalUnit.Mult1].s1 = instr.src1
            functional_unit_status.table[FunctionalUnit.Mult1].s1_fu = register_result_status.table[instr.src1]
            functional_unit_status.table[FunctionalUnit.Mult1].s1_ready = not functional_unit_status.table[FunctionalUnit.Mult1].s1_fu
            functional_unit_status.table[FunctionalUnit.Mult1].s2 = instr.src2
            functional_unit_status.table[FunctionalUnit.Mult1].s2_fu = register_result_status.table[instr.src2]
            functional_unit_status.table[FunctionalUnit.Mult1].s2_ready = not functional_unit_status.table[FunctionalUnit.Mult1].s2_fu
            register_result_status.table[instr.dst] = FunctionalUnit.Mult1
            is_issue = True
        elif (not functional_unit_status.table[FunctionalUnit.Mult2].busy) and (not register_result_status.table[instr.dst]):
            functional_unit_status.table[FunctionalUnit.Mult2].busy = True
            functional_unit_status.table[FunctionalUnit.Mult2].instr = instr
            functional_unit_status.table[FunctionalUnit.Mult2].op = "mult"
            functional_unit_status.table[FunctionalUnit.Mult2].dst = instr.dst
            functional_unit_status.table[FunctionalUnit.Mult2].s1 = instr.src1
            functional_unit_status.table[FunctionalUnit.Mult2].s1_fu = register_result_status.table[instr.src1]
            functional_unit_status.table[FunctionalUnit.Mult2].s1_ready = not functional_unit_status.table[FunctionalUnit.Mult2].s1_fu
            functional_unit_status.table[FunctionalUnit.Mult2].s2 = instr.src2
            functional_unit_status.table[FunctionalUnit.Mult2].s2_fu = register_result_status.table[instr.src2]
            functional_unit_status.table[FunctionalUnit.Mult2].s2_ready = not functional_unit_status.table[FunctionalUnit.Mult2].s2_fu
            register_result_status.table[instr.dst] = FunctionalUnit.Mult2
            is_issue = True
    
    elif type(instr) == SUBD:
        if (not functional_unit_status.table[FunctionalUnit.Add].busy) and (not register_result_status.table[instr.dst]):
            functional_unit_status.table[FunctionalUnit.Add].busy = True
            functional_unit_status.table[FunctionalUnit.Add].instr = instr
            functional_unit_status.table[FunctionalUnit.Add].op = "sub"
            functional_unit_status.table[FunctionalUnit.Add].dst = instr.dst
            functional_unit_status.table[FunctionalUnit.Add].s1 = instr.src1
            functional_unit_status.table[FunctionalUnit.Add].s1_fu = register_result_status.table[instr.src1]
            functional_unit_status.table[FunctionalUnit.Add].s1_ready = not functional_unit_status.table[FunctionalUnit.Add].s1_fu
            functional_unit_status.table[FunctionalUnit.Add].s2 = instr.src2
            functional_unit_status.table[FunctionalUnit.Add].s2_fu = register_result_status.table[instr.src2]
            functional_unit_status.table[FunctionalUnit.Add].s2_ready = not functional_unit_status.table[FunctionalUnit.Add].s2_fu
            register_result_status.table[instr.dst] = FunctionalUnit.Add
            is_issue = True
    
    elif type(instr) == ADDD:
        if (not functional_unit_status.table[FunctionalUnit.Add].busy) and (not register_result_status.table[instr.dst]):
            functional_unit_status.table[FunctionalUnit.Add].busy = True
            functional_unit_status.table[FunctionalUnit.Add].instr = instr
            functional_unit_status.table[FunctionalUnit.Add].op = "add"
            functional_unit_status.table[FunctionalUnit.Add].dst = instr.dst
            functional_unit_status.table[FunctionalUnit.Add].s1 = instr.src1
            functional_unit_status.table[FunctionalUnit.Add].s1_fu = register_result_status.table[instr.src1]
            functional_unit_status.table[FunctionalUnit.Add].s1_ready = not functional_unit_status.table[FunctionalUnit.Add].s1_fu
            functional_unit_status.table[FunctionalUnit.Add].s2 = instr.src2
            functional_unit_status.table[FunctionalUnit.Add].s2_fu = register_result_status.table[instr.src2]
            functional_unit_status.table[FunctionalUnit.Add].s2_ready = not functional_unit_status.table[FunctionalUnit.Add].s2_fu
            register_result_status.table[instr.dst] = FunctionalUnit.Add
            is_issue = True
            
    elif type(instr) == DIVD:
        if (not functional_unit_status.table[FunctionalUnit.Divide].busy) and (not register_result_status.table[instr.dst]):
            functional_unit_status.table[FunctionalUnit.Divide].busy = True
            functional_unit_status.table[FunctionalUnit.Divide].instr = instr
            functional_unit_status.table[FunctionalUnit.Divide].op = "div"
            functional_unit_status.table[FunctionalUnit.Divide].dst = instr.dst
            functional_unit_status.table[FunctionalUnit.Divide].s1 = instr.src1
            functional_unit_status.table[FunctionalUnit.Divide].s1_fu = register_result_status.table[instr.src1]
            functional_unit_status.table[FunctionalUnit.Divide].s1_ready = not functional_unit_status.table[FunctionalUnit.Divide].s1_fu
            functional_unit_status.table[FunctionalUnit.Divide].s2 = instr.src2
            functional_unit_status.table[FunctionalUnit.Divide].s2_fu = register_result_status.table[instr.src2]
            functional_unit_status.table[FunctionalUnit.Divide].s2_ready = not functional_unit_status.table[FunctionalUnit.Divide].s2_fu
            register_result_status.table[instr.dst] = FunctionalUnit.Divide
            is_issue = True
    
    if is_issue:
        instruction_status[instr].issue = cycle
        return True
    return False

In [52]:

pc = 0
cycle = 1
while True:
    if cycle > 62:
        break
    
    fus = wb(cycle)
    exec(cycle)
    read_oprands(cycle)
    if pc < len(instrs) and issue(instrs[pc], cycle):
        pc += 1
    if fus:
        release_delay(fus)
    
    print("Cycle: {}".format(cycle))
    cycle += 1
    
    functional_unit_status.display_table()
    display_instruction_status(instruction_status)
    register_result_status.display_table()


Cycle: 1
+------+---------+-------+------+----+----+----+----+----+------+------+
| Time |   Name  |  Busy |  Op  | Fi | Fj | Fk | Qj | Qk |  Rj  |  Rk  |
+------+---------+-------+------+----+----+----+----+----+------+------+
|      | Integer |  True | load | F6 |    | R2 |    |    | True | True |
|      |  Mult1  | False |      |    |    |    |    |    |      |      |
|      |  Mult2  | False |      |    |    |    |    |    |      |      |
|      |   Add   | False |      |    |    |    |    |    |      |      |
|      |  Divide | False |      |    |    |    |    |    |      |      |
+------+---------+-------+------+----+----+----+----+----+------+------+
+-------------------+-------+------+------+----+
|    Instructions   | issue | read | exec | wb |
+-------------------+-------+------+------+----+
|   L.D F6, 34(R2)  |   1   |      |      |    |
|   L.D F2, 45(R3)  |       |      |      |    |
|  MUL.D F0, F2, F4 |       |      |      |    |
|  SUB.D F8, F2, F6 |       |      |    